In [1]:
import pandas as pd
import plotly.plotly as py
import plotly.offline as offline
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import plotly.graph_objs as go

In [2]:
#Read datasets. In this case, we use the excell file of tags to link pumps/level
pump = pd.read_csv("/Users/sergiers/Desktop/CLASE/BDMA/TUe/Capita_selecta/waterschap-aa-en-maas_sewage/sewer_data/PUMPdata_pump_aarle-rixtel_flow-level_1.5years/flow/carpeta sin título/4140/GBS_AR.RG4140_Instellingen_FT510_AM_Debietmeting.Q.csv", delimiter=";", decimal=",", dtype={'Value': float})
level = pd.read_csv("/Users/sergiers/Desktop/CLASE/BDMA/TUe/Capita_selecta/waterschap-aa-en-maas_sewage/sewer_data/PUMPdata_pump_aarle-rixtel_flow-level_1.5years/flow/carpeta sin título/4140/xGBS_AR.RG4150_Instellingen_LT610_Niveaumeting.Q.csv", delimiter=";", decimal=",", dtype={'Value': float})
rain = pd.read_csv("/Users/sergiers/Desktop/CLASE/BDMA/TUe/Capita_selecta/waterschap-aa-en-maas_sewage/sewer_data/PUMPdata_pump_aarle-rixtel_flow-level_1.5years/flow/carpeta sin título/4140/4140-Mariahout.csv", delimiter=";", decimal=",", dtype={'Value': float})


In [3]:
#IMPORTANT: Parse dates
pump["TimeStamp"] = pd.to_datetime(pump['TimeStamp'], format='%d-%m-%Y %H:%M:%S')
level["TimeStamp"] = pd.to_datetime(level['TimeStamp'], format='%d-%m-%Y %H:%M:%S')
rain["Eind"] = pd.to_datetime(rain['Eind'])


In [4]:
#Create an index and remove timestamp column and tag name
level.index = level["TimeStamp"]
pump.index = pump["TimeStamp"]
rain.index = rain["Eind"]


In [5]:
level.index.names = ['Date']
pump.index.names = ['Date']
rain.index.names = ['Eind']

In [6]:
pump = pump[["Value"]]
level = level[["Value"]]
rain = rain[["VOLUME"]]

In [10]:
#NOT NECESARYCut the datasets:
#level = level.loc['2017-08-07 00:00:05':]
rain = rain.loc[:'2018-08-06']

In [13]:
#now we need to resample so data is homogeneous
level = level.resample('60T').mean()
pump = pump.resample('60T').mean()
rain = rain.resample('60T').mean()

In [14]:
flowdata = go.Scatter(
          x=pump.index,
          y=pump['Value'],
          name='flow')
leveldata = go.Scatter(
          x=level.index,
          y=level['Value'],
          name='Level',
          yaxis='y2')
raindata = go.Scatter(
          x=rain.index,
          y=rain['VOLUME'],
          name='Rain',
          yaxis='y3')
data = [flowdata,leveldata, raindata]

In [15]:
layout = go.Layout(
    title='Flow vs Level Vs Rain Aarle 4140',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label='1d',
                     step='day',
                     stepmode='backward'),
                dict(count=7,
                     label='1w',
                     step='day',
                     stepmode='backward'),
                dict(count=1,
                     label='1m',
                     step='month',
                     stepmode='backward'),
                dict(step='all')
            ])
        ),        
        rangeslider=dict(
            visible = True
        ),
        type='date'
    ),
    yaxis=dict(
        title='Flow (m3/h)'
    ),
    yaxis2=dict(
        title='Level (m above NAP)',
        overlaying='y',
        side='right'
    ),
    yaxis3=dict(
        title='Rain Volume',
        overlaying='y',
        side='right'
    )
)
fig = dict(data=data, layout=layout)
offline.plot(fig, filename='4140.html')
#py.iplot(fig)

'file:///Users/sergiers/PycharmProjects/DATARESPONSIBLECHALLENGE/Image_Related_analysis/Flow-VS-level-AarleRixtel/4140.html'